In [4]:
import random
import pandas as pd
import mysql.connector

In [5]:
def connect_to_mysql(host='localhost', user='shardul', password='password', database='CHATDB'):
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        return connection, connection.cursor()
    except mysql.connector.Error as err:
        print(f"MySQL Connection Error: {err}")
        return None, None

In [ ]:
def detect_pattern(user_input):
    user_input = user_input.lower()

    if "highest" in user_input and "sales" in user_input and "store" in user_input:
        return "total_store_sales"
    elif "example" in user_input and "group by" in user_input:
        return "example_grp_by"
    else:
        return "example"

In [12]:
def generate_query(pattern, params):

    if pattern == "example":
        rnd = random.randint(0, 9)
        sql_queries = [
            "SELECT * FROM CoffeeSales WHERE store_location = 'Lower Manhattan';",
            "SELECT * FROM CoffeeSales WHERE product_category = 'Coffee';",
            "SELECT * FROM CoffeeSales WHERE transaction_date = '1/1/2023';",
            "SELECT * FROM CoffeeSales WHERE transaction_qty > 2;",
            "SELECT * FROM CoffeeSales WHERE unit_price > 3.00;",
            "SELECT * FROM CoffeeSales WHERE product_type = 'Organic brewed coffee';",
            "SELECT * FROM CoffeeSales WHERE product_id = 79;",
            "SELECT * FROM CoffeeSales WHERE product_category = 'Tea';",
            "SELECT * FROM CoffeeSales WHERE product_detail LIKE '%Green Tea%';",
            "SELECT * FROM CoffeeSales WHERE unit_price BETWEEN 2.00 AND 4.00;"
        ]

        return {"SQL": sql_queries[rnd], "MongoDB": None}
    elif pattern == "example_grp_by":
        return {"SQL": None, "MongoDB": None}
    elif pattern == "total_store_sales":
        sql_query = "SELECT store_id, store_location, ROUND(SUM(transaction_qty * unit_price), 2) AS total_store_sales, COUNT(DISTINCT transaction_id) AS total_transactions FROM CoffeeSales GROUP BY store_id,store_location ORDER BY total_store_sales DESC LIMIT 5;"
        mongo_query = {
            "$group": { "_id": "$Description", "totalSales": { "$sum": { "$multiply": ["$Quantity", "$UnitPrice"] } } }
        }
        return {"SQL": sql_query, "MongoDB": mongo_query}

    return None

In [13]:
def run_sql_query(sql_query,cursor):
    cursor.execute(sql_query)
    results = cursor.fetchall()
    return results

In [14]:
def process_user_input(user_input):
    # Detect pattern
    pattern = detect_pattern(user_input)
    if not pattern:
        return "No matching pattern found."

    # Extract parameters
    params = {}
    if "quantity greater" in user_input:
        quantity = int(user_input.split("greater than")[-1].strip())
        params["quantity"] = quantity
    elif "top" in user_input:
        limit = int(user_input.split("top")[-1].split()[0].strip())
        params["limit"] = limit
    elif "date" in user_input:
        date = user_input.split("date")[-1].strip()
        params["date"] = date
    elif "customer" in user_input:
        customer_id = int(user_input.split("customer")[-1].strip())
        params["customer_id"] = customer_id

    # Generate and execute query
    generated_query = generate_query(pattern, params)
    if generated_query and "SQL" in generated_query:
        sql_query = generated_query["SQL"]

        connection, cursor = connect_to_mysql()
        if connection and cursor:
            try:
                result = run_sql_query(sql_query,cursor);
            finally:
                cursor.close()
                connection.close()
        return sql_query, result
    
    return generated_query

In [15]:
# Test the system with an example input
user_input = "top 5 stores by total sales"
sql_query, result = process_user_input(user_input)
print("Generated SQL Query:\n", sql_query)
print("\nQuery Result:")
print(result)

Generated SQL Query:
 SELECT store_id, store_location, ROUND(SUM(transaction_qty * unit_price), 2) AS total_store_sales, COUNT(DISTINCT transaction_id) AS total_transactions FROM CoffeeSales GROUP BY store_id,store_location ORDER BY total_store_sales DESC LIMIT 5;

Query Result:
[(8, "Hell's Kitchen", Decimal('15714.55'), 3441), (3, 'Astoria', Decimal('15591.05'), 3385), (5, 'Lower Manhattan', Decimal('14966.12'), 3174)]
